# Danish fire insurance data

In [21]:
import sys
sys.path.insert(0,'/home/pgoffard/BayesSplicedModels/')
%run ../../preamble.py
danish = pd.read_csv("../../Data/Danish/danish.csv").x
import random
random.seed(123)

# Martin and Hansjoerg method

In [22]:
X = danish.values
n, k = len(X), 30
def compute_Tk(X, k):
    n = len(X)
    X_sorted = np.sort(X)
    Y_k = np.flip(X_sorted[n-k:] / X_sorted[n-1-k])
    num = np.cumsum(np.log(Y_k))/np.arange(1, k+1, 1)
    denom = 1 + np.flip(np.cumsum(np.flip(np.append(1/np.arange(2, k+1, 1), 0))))
    Tk = num / denom 
    return(Tk)

def fp(p):
    res = (1-np.exp(1-2*p)*(1-2*p)*sp.exp1(1-2*p)-np.exp(2-2*p)*sp.exp1(1-p)**2) / p**2/ (1-p)**2 + \
    2*(np.exp(2-p)*sp.exp1(1-p)*sp.exp1(1)-1+np.exp(1-p)*(1-p)*sp.exp1(1-p)) / p**2 / (1-p) + \
    (1-np.exp(1)*sp.exp1(1)-np.exp(2)*sp.exp1(1)**2) / p**2
    return(res)


def optimal_threshold(X, p = -1):
    n = len(X)
    X_sorted = np.sort(X)
    C = 0.502727
    emp_var = np.array([np.var(compute_Tk(X, k)) for k in np.arange(2, len(X) - int(len(X)/5), 1)])
    k_ast = np.where(emp_var == np.min(emp_var))[0][0]
    k0_ast = k_ast * (C / fp(p)/ (1-p)**2)**(-1/(1-2*p))
    return(X_sorted[n-1-int(k0_ast)]) 

optimal_threshold(X, p = -1)

2.781844802

# Fitting splicing models to the data with fixed threshold

In [15]:
# The data
X = danish.values
# Model for the bulk distribution
body_model_names = ["Log-Logistic", "Lomax", "Burr", "Exp", "Gamma", "Weibull", "Inverse-Gaussian", "Lognormal", "Inverse-Weibull"]
body_model_param_names = [["β1", "σ1"], ["α1", "σ1"], ["α1","β1", "σ1"], ["λ1"], ["r1", "m1"], ["k1", "β1"],["μ1", "λ1"],
                          ["μ1", "σ1"], ["k1", "β1"]]

# Prior distributions over the parameters of the bulk distribution
body_model_priors= [
                [bsm.prior_model('gamma',body_model_param_names[0][0], 1, 1), 
                 bsm.prior_model('gamma',body_model_param_names[0][1], 1, 1)],
    [bsm.prior_model('gamma',body_model_param_names[1][0], 1, 1), 
     bsm.prior_model('gamma',body_model_param_names[1][1], 1, 1)],
    [bsm.prior_model('gamma',body_model_param_names[2][0], 1, 1), 
     bsm.prior_model('gamma',body_model_param_names[2][1], 1, 1),
     bsm.prior_model('gamma',body_model_param_names[2][2], 1, 1)], 
    [bsm.prior_model('gamma',body_model_param_names[3][0], 1, 1)], 
     [bsm.prior_model('gamma',body_model_param_names[4][0], 1, 1),
     bsm.prior_model('gamma',body_model_param_names[4][1], 1, 1)],
    [bsm.prior_model('gamma',body_model_param_names[5][0], 1, 1),
     bsm.prior_model('gamma',body_model_param_names[5][1], 1, 1)],
    [bsm.prior_model('gamma',body_model_param_names[6][0], 1, 1),
     bsm.prior_model('gamma',body_model_param_names[6][1], 1, 1)],
    [bsm.prior_model('normal',body_model_param_names[7][0], 0, 0.5),
     bsm.prior_model('gamma',body_model_param_names[7][1], 1, 1)],
     [bsm.prior_model('gamma',body_model_param_names[8][0], 1, 1),
     bsm.prior_model('gamma',body_model_param_names[8][1], 1, 1)]
]
# Model for the tail of the distribution
tail_model_names = ["Pareto-Tail", "GPD-Tail"]


tail_model_param_names = [ ["α2"], ["ξ2","σ2"]]

# Prior distributions over the parameters of the bulk distribution
tail_model_priors= [
                [bsm.prior_model('gamma',tail_model_param_names[0][0], 1, 1)],
                [bsm.prior_model('gamma',tail_model_param_names[1][0], 1, 1), bsm.prior_model('gamma',tail_model_param_names[1][1], 1, 1)]
]
             
γ_prior, p_prior = bsm.prior_model('fixed', "γ", optimal_threshold(X, p = -1), max(X)), bsm.prior_model('uniform',"p", 0, 1)

#Splicing model type
splicing_types = ["disjoint", "simple", "continuous"]

# Setting the models
fs, f_names, prior_spliced_model = [], [], []
for i in range(len(body_model_names)):
    for j in range(len(tail_model_names)):
        for splicing_type in splicing_types:
            f1, f2 =  bsm.loss_model(body_model_names[i], body_model_param_names[i]), bsm.loss_model(tail_model_names[j], tail_model_param_names[j])
            fs.append(bsm.spliced_loss_model(f1 , f2, splicing_type))
            f_names.append(body_model_names[i] +"_"+ tail_model_names[j]+"_"+splicing_type)
            if splicing_type == "disjoint": 
                prior_spliced_model.append(bsm.independent_priors(body_model_priors[i] + tail_model_priors[j] + [γ_prior, p_prior]))
            else:
                prior_spliced_model.append(bsm.independent_priors(body_model_priors[i] + tail_model_priors[j] + [γ_prior]))  
for f in fs:
    f.set_ppf()
f_spliced_dic = dict(zip(f_names, fs))

In [16]:
splits = 2
len(fs)/ splits

27.0

In [ ]:
popSize, ρ, c, n_step_max, err, paralell, n_proc, verbose = 10000, 1/2, 0.99, 25, 1e-6, False, 4, False
def fit_spliced_models(i):
    print(f_names[i])
    trace, log_marg, DIC, WAIC = bsm.smc_likelihood_annealing(X, fs[i], popSize, prior_spliced_model[i], ρ, c,n_step_max, err, paralell, 4, verbose)
    return([trace, log_marg, DIC, WAIC])

splits, res = 2, []
for k in range(splits):
    print("Model batch #"+str(k))
    %time res_sub = Parallel(n_jobs=int(len(fs)/splits))(delayed(fit_spliced_models)(i) for i in range(int(k * len(f_names) / splits), int((k+1) * len(f_names)/splits)))
    res += res_sub


Model batch #0


In [ ]:
fit_spliced_models_dic = dict(zip(f_names, res))
γ_map = np.array([fit_spliced_models_dic[f_names[k]][0]['γ'].mean() for k in range(len(fit_spliced_models_dic))])
spliced_model_df = pd.DataFrame({'model':f_names,
                                 "d": np.array([f.d for f in fs]),
                                 "γ_map": np.array([fit_spliced_models_dic[f_names[k]][0]['γ'].mean() for k in range(len(fit_spliced_models_dic))]),
                                 'log_marg':  np.array([fit_spliced_models_dic[f_names[k]][1] for k in range(len(fit_spliced_models_dic))]), 
                                 "DIC": np.array([fit_spliced_models_dic[f_names[k]][2] for k in range(len(fit_spliced_models_dic))]), 
                                 "WAIC":np.array([fit_spliced_models_dic[f_names[k]][3] for k in range(len(fit_spliced_models_dic))])})
spliced_model_df["posterior_probability"] = np.exp(spliced_model_df["log_marg"] - np.max(spliced_model_df["log_marg"])) / np.sum(np.exp(spliced_model_df["log_marg"] - np.max(spliced_model_df["log_marg"]))) 

spliced_model_df["Wass_dist"] = np.array([bsm.compute_Wasserstein(X, f_spliced_dic[model_name], fit_spliced_models_dic[model_name][0].mean().values, 1) for model_name in spliced_model_df["model"].values])
spliced_model_df.sort_values(by='log_marg', ascending=False)

In [ ]:
model_names = spliced_model_df.sort_values(by='Wass_dist', ascending=True)["model"][:6]
for model_name in model_names:
    f, trace = f_spliced_dic[model_name], fit_spliced_models_dic[model_name][0]
    bsm.posterior_plots(f, trace)
#     bsm.trace_plots(f, trace)
#     bsm.qq_plot(X, f, trace.mean().values)


In [ ]:
import pickle
spliced_model_df.to_csv("../../Data/Danish/danish_spliced_models_fixed_threshold.csv", sep=',')
with open('../../Data/Danish/trace_danish_spliced_models_fixed_threshold', 'wb') as fp:
    pickle.dump(fit_spliced_models_dic, fp)